In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Admin\\Desktop\\code\\python\\Mlops-Production-Ready-Deep-Learning-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Admin\\Desktop\\code\\python\\Mlops-Production-Ready-Deep-Learning-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    testing_data: Path | None
    all_params: dict
    mlflow_uri: int
    params_image_size: list
    params_batch_size: int





In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [8]:
class ConfigurationManager:
    def __init__(self,config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH):
        self.Config = read_yaml(config_file_path)
        self.Params = read_yaml(params_file_path)
         
        create_directories([self.Config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        training = self.Config.training
        model_uri = self.Config.evaluation.model_uri

        evaluation_config = EvaluationConfig(
            path_of_model= Path(training.trained_model_path),
            training_data= os.path.join(self.Config.data_ingestion.unzip_dir,"Data", "train"),
            testing_data=  None, #os.path.join(self.Config.data_ingestion.unzip_dir,"Data","test"), # None
            mlflow_uri= model_uri,
            all_params= self.Params,
            params_batch_size= self.Params.BATCH_SIZE,
            params_image_size=self.Params.IMAGE_SIZE

        )

        return evaluation_config
            


In [11]:
import tensorflow as tf
from pathlib import Path
from cnnClassifier.utils.common import save_json
import mlflow
import mlflow.keras
from urllib.parse import urlparse

import dagshub


In [15]:
dagshub.init(repo_owner='harcuracy', repo_name='Mlops-Production-Ready-Deep-Learning-Project', mlflow=True)

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config


    def _valid_generator(self):

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:2],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
            shuffle=False
        )

        #CASE 1: Use separate testing data
        if self.config.testing_data is not None:
            valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale=1. / 255
            )

            self.valid_generator = valid_datagenerator.flow_from_directory(
                directory=self.config.testing_data,
                **dataflow_kwargs
            )

        #CASE 2: Use validation split from training data
        else:
            valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale=1. / 255,
                validation_split=0.30
            )

            self.valid_generator = valid_datagenerator.flow_from_directory(
                directory=self.config.training_data,
                subset="validation",
                **dataflow_kwargs
            )



    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)



    def evaluate(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)


    def save_score(self):
        scores = {"loss":self.score[0],"accuracy":self.score[1]}
        save_json(path= Path("scores.json"),data= scores)


    def log_into_mlflow(self):
        # Set DagsHub as tracking and registry
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        mlflow.set_registry_uri(self.config.mlflow_uri)
        
        #Set experiment
        mlflow.set_experiment("chest cancer experiment")
        
        with mlflow.start_run():
            # log dataset type
            mlflow.log_param(
                "evaluation_dataset",
                "testing" if self.config.testing_data else "validation_split"
            )
            
            # log params and metrics
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1]
            })
            
            # log model with a name
            mlflow.keras.log_model(
                self.model,
                "model",
                registered_model_name="resnet50Model"
            )


    

    
        
                                                                           


[2026-01-19 15:53:18,324: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as harcuracy

[2026-01-19 15:53:18,349: INFO: helpers: Accessing as harcuracy]
[2026-01-19 15:53:19,945: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/harcuracy/Mlops-Production-Ready-Deep-Learning-Project "HTTP/1.1 200 OK"]
[2026-01-19 15:53:21,350: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "harcuracy/Mlops-Production-Ready-Deep-Learning-Project"

[2026-01-19 15:53:21,355: INFO: helpers: Initialized MLflow to track repo "harcuracy/Mlops-Production-Ready-Deep-Learning-Project"]


Repository harcuracy/Mlops-Production-Ready-Deep-Learning-Project initialized!

[2026-01-19 15:53:21,356: INFO: helpers: Repository harcuracy/Mlops-Production-Ready-Deep-Learning-Project initialized!]


In [16]:
try:
    config = ConfigurationManager()
    evaluation_config = config.get_evaluation_config()
    evaluation = Evaluation(config = evaluation_config)
    evaluation.evaluate()
    evaluation.save_score()
    evaluation.log_into_mlflow()
    
 
    


except Exception as e:
    raise e

[2026-01-19 15:53:37,995: INFO: common: yaml file: config\config.yaml loaded successfully]


[2026-01-19 15:53:38,019: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-19 15:53:38,021: INFO: common: created directory at: artifacts]
Found 182 images belonging to 4 classes.
12/12 [==============================] - 14s 1s/step - loss: 1.7801 - accuracy: 0.2747
[2026-01-19 15:53:54,668: INFO: common: json file saved at: scores.json]


2026/01/19 15:53:58 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2026-01-19 15:54:10,692: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 54). These functions will not be directly callable after loading.]
[2026-01-19 15:54:14,760: INFO: builder_impl: Assets written to: C:\Users\Admin\AppData\Local\Temp\tmpvjbiuaww\model\data\model\assets]


2026/01/19 15:54:35 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2023-02-28; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'backports-tarfile'}
c:\Users\Admin\Desktop\code\python\Mlops-Production-Ready-Deep-Learning-Project\.venv\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
2026/01/19 15:54:35 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


[2026-01-19 15:58:46,660: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=5, redirect=5, status=5)) after connection broken by 'SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2436)')': /harcuracy/Mlops-Production-Ready-Deep-Learning-Project.mlflow/api/2.0/mlflow-artifacts/artifacts/05cfb21518294f10aa69f3cc04dda810/e6dfccceb4c2487d98c85fe5aa5b2862/artifacts/model/data/model/variables/variables.data-00000-of-00001]


KeyboardInterrupt: 